In [1]:
%pylab inline

import math
import numpy as np
from scipy.sparse.linalg import inv
#from numpy.linalg import inv
import scipy.sparse as sps
import scipy.sparse.linalg
from scipy import integrate
import sys
import matplotlib.pyplot as plt
sys.path.append('../../src/')
from pylab import *
import shutil

import parameters as pam
import lattice as lat
import variational_space as vs
import hamiltonian as ham
import basis_change as basis
import get_state as getstate
import utility as util
import plotfig as fig
import ground_state as gs
import lanczos
import time
start_time = time.time()
M_PI = math.pi
                  
#####################################
def compute_Aw_main(ANi,ACu,epCu,epNi,epbilayer,tpd,tpp,tapzd,tapzp,tz_a1a1,tz_b1b1,pds,pdp,pps,ppp,Upp,Uss,\
                    d_Ni_double,d_Cu_double,p_double,apz_double,double_Ni_part,hole345_Ni_part, double_Cu_part,\
                    hole345_Cu_part, idx_Ni,idx_Cu, U_Ni, \
                    S_Ni_val, Sz_Ni_val, AorB_Ni_sym, \
                    U_Cu, S_Cu_val, Sz_Cu_val, AorB_Cu_sym):  
    if Norb==8 or Norb==5:
        fname = 'epCu'+str(epCu)+'epNi'+str(epNi)+'_tpd'+str(tpd)+'_tpp'+str(tpp) \
                  +'_Mc'+str(Mc)+'_Norb'+str(Norb)+'_eta'+str(eta) +'_ANi'+str(ANi) \
                  + '_ACu'+str(ACu) + '_B'+str(B) + '_C'+str(C) +'_tz_a1a1' +str(tz_a1a1)  +'_tz_b1b1' +str(tz_b1b1)                 
        flowpeak = 'Norb'+str(Norb)+'_tpp'+str(tpp)+'_Mc'+str(Mc)+'_eta'+str(eta)
    elif Norb==10 or Norb==11 or Norb==12:
        fname = 'epCu'+str(epCu)+'epNi'+str(epNi)+'_pdp'+str(pdp)+'_pps'+str(pps)+'_ppp'+str(ppp) \
                  +'_Mc'+str(Mc)+'_Norb'+str(Norb)+'_eta'+str(eta) \
                  +'_ANi'+str(ANi) + '_ACu'+str(ACu) + '_B'+str(B) + '_C'+str(C)
        flowpeak = 'Norb'+str(Norb)+'_pps'+str(pps)+'_ppp'+str(ppp)+'_Mc'+str(Mc)+'_eta'+str(eta) 
               
                
    w_vals = np.arange(pam.wmin, pam.wmax, pam.eta)
    Aw = np.zeros(len(w_vals))
    Aw_dd_total = np.zeros(len(w_vals))
    Aw_d8_total = np.zeros(len(w_vals))

    # set up H0
    if Norb==8 or Norb==5:
        tpd_nn_hop_dir, tpd_orbs, tpd_nn_hop_fac, tpp_nn_hop_fac \
                                   = ham.set_tpd_tpp(Norb,tpd,tpp,0,0,0,0)
    elif Norb==10 or Norb==12:
        tpd_nn_hop_dir, tpd_orbs, tpd_nn_hop_fac, tpp_nn_hop_fac \
                                   = ham.set_tpd_tpp(Norb,0,0,pds,pdp,pps,ppp)
        
    if Norb==5:  
        tapzd_nn_hop_dir, tapzd_orbs, tapzd_nn_hop_fac, tapzp_nn_hop_dir, tapzp_orbs, tapzp_nn_hop_fac\
                                   = ham.set_tdO_tpO(Norb,tapzd,tapzp)       
        
    
    tz_fac = ham.set_tz(Norb,if_tz_exist,tz_a1a1,tz_b1b1)
            
    T_pd   = ham.create_tpd_nn_matrix(VS,tpd_nn_hop_dir, tpd_orbs, tpd_nn_hop_fac)
    T_pp   = ham.create_tpp_nn_matrix(VS,tpp_nn_hop_fac)  
    T_z    = ham.create_tz_matrix(VS,tz_fac)
    T_apzd   = ham.create_tapzd_nn_matrix(VS,tapzd_nn_hop_dir, tapzd_orbs, tapzd_nn_hop_fac)
    T_apzp   = ham.create_tapzp_nn_matrix(VS,tapzp_nn_hop_dir, tapzp_orbs, tapzp_nn_hop_fac)    
    Esite  = ham.create_edep_diag_matrix(VS,ANi,ACu,epNi,epCu,epbilayer)      
    
    H0 = T_pd + T_pp + T_z + T_apzd + T_apzp + Esite    
#     H0 = T_pd   
    print("H0 %s seconds ---" % (time.time() - start_time))   
            
    '''
    Below probably not necessary to do the rotation by multiplying U and U_d
    the basis_change.py is only for label the state as singlet or triplet
    and assign the interaction matrix
    '''
    if pam.if_H0_rotate_byU==1:
        H0_Ni_new = U_Ni_d.dot(H0.dot(U_Ni))  
    

    if Norb==5 or Norb==6 or Norb==10 or Norb==11 or Norb==12:     
        Hint_Ni = ham.create_interaction_matrix_ALL_syms(VS,d_Ni_double,p_double,apz_double,double_Ni_part, idx_Ni, hole345_Ni_part,  \
                                                      S_Ni_val, Sz_Ni_val,AorB_Ni_sym, ACu, ANi, Upp, Uss)
        Hint_Cu = ham.create_interaction_matrix_ALL_syms(VS,d_Cu_double,p_double,apz_double,double_Cu_part, idx_Cu, hole345_Cu_part, \
                                                      S_Cu_val, Sz_Cu_val,AorB_Cu_sym, ACu, ANi, Upp, Uss)        
        
        if pam.if_H0_rotate_byU==1:
            H_Ni = H0_Ni_new + Hint_Ni
            
            # continue rotate the basis for setting Cu layer's interaction (d_Cu_double)
            H0_Cu_new = U_Cu_d.dot(H_Ni.dot(U_Cu)) 
            H = H0_Cu_new + Hint_Cu
        else:
            H = H0 + Hint_Ni + Hint_Cu

        print("interaction %s seconds ---" % (time.time() - start_time))               
#         if pam.basis_change_type=='all_states':
#             U_other, S_Ni_val, Sz_Ni_val, S_Cu_val, Sz_Cu_val \
#                 = basis.create_singlet_triplet_basis_change_matrix_other_states(VS, d_Ni_double, d_Cu_double)
#             U_other_d = (U_other.conjugate()).transpose()  
            
#             H = U_other_d.dot(H.dot(U_other))
        H_bond = U_bond_d.dot(H.dot(U_bond))  
    
    
        H_bond.tocsr()

        ####################################################################################
        # compute GS only for turning on full interactions
        if pam.if_get_ground_state==1:
            vals, vecs = gs.get_ground_state(H_bond, VS, S_Ni_val,Sz_Ni_val,S_Cu_val,Sz_Cu_val,bonding_val)
        print("ground state %s seconds ---" % (time.time() - start_time))                   
                
#             if Norb==8:
#                 util.write_GS('Egs_'+flowpeak+'.txt',A,ep,tpd,vals[0])
#             elif Norb==10 or Norb==11 or Norb==12:
#                 util.write_GS2('Egs_'+flowpeak+'.txt',A,ep,pds,pdp,vals[0])
        #########################################################################
        '''
        Compute A(w) for various states
        '''
        if pam.if_compute_Aw==1:
            clf()
#             # compute d8
#             fig.compute_Aw_d8_sym(H, VS, d_double, S_val, Sz_val, AorB_sym, A, w_vals, "Aw_d8_sym_", fname)

            #compute d9Ld9L
            d9Ld9L_a1L_b1L_state_indices, d9Ld9L_a1L_b1L_state_labels, \
                    = getstate.get_d9Ld9L_state_indices(VS)
            fig.compute_Aw1(H, VS, w_vals,  d9Ld9L_a1L_b1L_state_indices, d9Ld9L_a1L_b1L_state_labels, "Aw_d9Ld9L_a1L_b1L__", fname)
           
        
#             #compute d9d9L2        
#             d9d9L2_a1_b1L2_state_indices, d9d9L2_a1_b1L2_state_labels, \
#                     = getstate.get_d9d9L2_state_indices(VS)
#             fig.compute_Aw1(H, VS, w_vals,  d9d9L2_a1_b1L2_state_indices, d9d9L2_a1_b1L2_state_labels, "Aw_d9d9L2_a1_b1L2__", fname)
#             #compute d9L2d9        
#             d9L2d9_a1L2_b1_state_indices, d9L2d9_a1L2_b1_state_labels, \
#                     = getstate.get_d9L2d9_state_indices(VS)
#             fig.compute_Aw1(H, VS, w_vals,  d9L2d9_a1L2_b1_state_indices, d9L2d9_a1L2_b1_state_labels, "Aw_d9L2d9_a1L2_b1__", fname)
                    
#             fig.compute_Aw_d8d8_sym(H, VS, d_Ni_double, S_Ni_val, Sz_Ni_val, AorB_Ni_sym, ANi, S_Cu_val, \
#                                     Sz_Cu_val,AorB_Cu_sym, ACu, w_vals, "Aw_d8d8_sym_", fname)    
##########################################################################
if __name__ == '__main__': 
    Mc  = pam.Mc
    print ('Mc=',Mc)

    Norb = pam.Norb
    eta  = pam.eta
    edNi = pam.edNi
    edCu = pam.edCu

    ANis = pam.ANis
    ACus = pam.ACus
    B  = pam.B
    C  = pam.C
    
    tz_a1a1 = pam.tz_a1a1
    tz_b1b1 = pam.tz_b1b1    
    
    if_tz_exist  = pam.if_tz_exist    
    
    # set up VS
    VS = vs.VariationalSpace(Mc)
#     basis.count_VS(VS)
    print("vs %s seconds ---" % (time.time() - start_time))      
    
    d_Ni_double, idx_Ni, hole345_Ni_part,  double_Ni_part, \
    d_Cu_double, idx_Cu, hole345_Cu_part,  double_Cu_part, \
    p_double,apz_double = ham.get_double_occu_list(VS)
    
    # change the basis for d_double states to be singlet/triplet
    
    if pam.basis_change_type =='all_states':
        U_Ni,S_Ni_val, Sz_Ni_val, AorB_Ni_sym,\
                                        =  basis.create_singlet_triplet_basis_change_matrix \
                                        (VS, double_Ni_part, idx_Ni, hole345_Ni_part,d_Ni_double, d_Cu_double, 'Ni')
          
        U_Cu,S_Cu_val, Sz_Cu_val, AorB_Cu_sym,\
                                        =  basis.create_singlet_triplet_basis_change_matrix \
                                        (VS, double_Cu_part, idx_Cu, hole345_Cu_part,d_Ni_double, d_Cu_double, 'Cu')
 
    if pam.basis_change_type =='d_double':
        print("2 %s seconds ---" % (time.time() - start_time))           
        U_Ni,S_Ni_val, Sz_Ni_val, AorB_Ni_sym,\
                     =  basis.create_singlet_triplet_basis_change_matrix_d_double(VS, d_Ni_double, double_Ni_part, idx_Ni, hole345_Ni_part)
        print("3 %s seconds ---" % (time.time() - start_time))          
        U_Cu,S_Cu_val, Sz_Cu_val, AorB_Cu_sym,\
                     =  basis.create_singlet_triplet_basis_change_matrix_d_double(VS, d_Cu_double, double_Cu_part, idx_Cu, hole345_Cu_part)
        
        
    print("1 %s seconds ---" % (time.time() - start_time))            
    U_bond,bonding_val = basis.create_bonding_anti_bonding_basis_change_matrix(VS)    
        
        
    print("basis %s seconds ---" % (time.time() - start_time))    
        
    if pam.if_print_VS_after_basis_change==1:
        basis.print_VS_after_basis_change(VS,S_val,Sz_val)
            
#         U = U_Ni+U_Cu
#         print(U_Ni)
#         print(U_Cu)    
    U_Ni_d = (U_Ni.conjugate()).transpose()
    U_Cu_d = (U_Cu.conjugate()).transpose()    
    U_bond_d = (U_bond.conjugate()).transpose()        
    
    # check if U if unitary
#     util.checkU_unitary(U_Ni,U_Ni_d)
    
    if Norb==8 or Norb==5:
            for tpd in pam.tpds:
                for tapzd in pam.tapzds: 
                    for tapzp in pam.tapzps:
                        for epCu in pam.epCus:
                            for epNi in pam.epNis: 
                                for epbilayer in pam.epbilayers:                         
                                    for ANi in pam.ANis:
                                        for ACu in pam.ACus:
                #                            util.get_atomic_d8_energy(ANi,B,C)
                                            for tpp in pam.tpps:
                                                for Upp in pam.Upps:
                                                    for Uss in pam.Usss:
                                                        print ('===================================================')
                                                        print ('ANi=',ANi, 'ACu=',ACu,'epCu=', epCu, 'epNi=',epNi,\
                                                               ' tpd=',tpd,' tpp=',tpp,' Upp=',Upp,' Uss=',Uss ,'tz_a1a1=',tz_a1a1,'tz_b1b1=',tz_b1b1,\
                                                               'tapzd=',tapzd,'tapzp=',tapzp)

                                                        compute_Aw_main(ANi,ACu,epCu,epNi,epbilayer,tpd,tpp,tapzd,tapzp,tz_a1a1,tz_b1b1,0,0,0,0,Upp,Uss,\
                                                                        d_Ni_double,d_Cu_double,p_double,apz_double,double_Ni_part,hole345_Ni_part,\
                                                                        double_Cu_part,hole345_Cu_part, idx_Ni,idx_Cu, \
                                                                        U_Ni, S_Ni_val, Sz_Ni_val, AorB_Ni_sym ,U_Cu, \
                                                                        S_Cu_val, Sz_Cu_val, AorB_Cu_sym)  

                        
    print("--- %s seconds ---" % (time.time() - start_time))

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib
Ni_Cu_orbs =  ['d3z2r2', 'dx2y2']
O1_orbs =  ['px']
O2_orbs =  ['py']
Obilayer_orbs =  ['apz']
compute A(w) for symmetries =  ['1A1', '3B1', '3B1', '1A2', '3A2', '1E', '3E']
Mc= 2
VS.dim =  43164
vs 123.48832082748413 seconds ---
len(d_Ni_list) 6360
len(d_Cu_list) 6360
len(p_list) 12608
len(apz_list) 1576
len(idx_Ni) 6360
len(idx_Cu) 6360
[56, 57, 80, 81, 92, 93, 94, 95, 96, 97, 98, 99, 350, 351, 424, 425, 426, 427, 428, 429, 452, 453, 486, 487, 488, 489, 490, 491, 492, 493, 494, 495, 496, 497, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 557, 558, 559, 560, 561, 562, 563, 564, 565, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577, 578, 579, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 598, 599, 600, 60

basis _singlet_triplet_2.011831521987915 seconds ---
3 125.8379967212677 seconds ---
basis _singlet_triplet_1.9654500484466553 seconds ---
1 127.81186842918396 seconds ---
basis _bonding_anti_bonding_1.3123936653137207 seconds ---
basis 129.13276839256287 seconds ---
ANi= 6.0 ACu= 6.0 epCu= 3.2 epNi= 3.2  tpd= 1.2  tpp= 0.35  Upp= 0  Uss= 0 tz_a1a1= 0.044 tz_b1b1= 0.047 tapzd= 2.4 tapzp= 0.0
start create_tpd_nn_matrix
start create_tpp_nn_matrix
start create_tz_matrix
start create_tapzd_nn_matrix
start create_tapzp_nn_matrix
start create_edep_diag_matrix
H0 156.7031011581421 seconds ---
start create_interaction_matrix
--- create_interaction_matrix_ALL_syms 0.782665491104126 seconds ---
start create_interaction_matrix
--- create_interaction_matrix_ALL_syms 0.7736716270446777 seconds ---
interaction 158.29764914512634 seconds ---
start getting ground state
lowest eigenvalue of H from np.linalg.eigsh = 
[7.88216722 7.88216722 7.88216722 7.88216722 7.90976699 7.90976699
 8.15304711 8.153047

eigenvalue =  7.882167216569529
k =  1
Compute the weights in GS (lowest Aw peak)
 state  34215   d3z2r2 dn 0 0 2   dx2y2 dn 0 0 2   d3z2r2 up 0 0 0   dx2y2 up 0 0 0   px dn 1 0 0 
 S_Ni= 1 ,  Sz_Ni= -1 ,  S_Cu= 1 ,  Sz_Cu= 1 ,  bonding= -1 , weight =  0.0395519001651705 

 state  12449   d3z2r2 up 0 0 2   dx2y2 up 0 0 2   d3z2r2 dn 0 0 0   dx2y2 dn 0 0 0   px dn -1 0 2 
 S_Ni= 1 ,  Sz_Ni= 1 ,  S_Cu= 1 ,  Sz_Cu= -1 ,  bonding= -1 , weight =  0.039551900165170316 

 state  23431   d3z2r2 dn 0 0 2   dx2y2 dn 0 0 2   d3z2r2 up 0 0 0   dx2y2 up 0 0 0   py dn 0 1 0 
 S_Ni= 1 ,  Sz_Ni= -1 ,  S_Cu= 1 ,  Sz_Cu= 1 ,  bonding= -1 , weight =  0.039551900165170316 

 state  12673   d3z2r2 up 0 0 2   dx2y2 up 0 0 2   d3z2r2 dn 0 0 0   dx2y2 dn 0 0 0   py dn 0 -1 2 
 S_Ni= 1 ,  Sz_Ni= 1 ,  S_Cu= 1 ,  Sz_Cu= -1 ,  bonding= -1 , weight =  0.03955190016517015 

 state  12870   d3z2r2 dn 0 0 2   dx2y2 dn 0 0 2   dx2y2 up 0 0 0   dx2y2 dn 0 0 0   apz up 0 0 1 
 S_Ni= 1 ,  Sz_Ni= -1 ,  S_Cu= 0 ,  Sz_Cu= 0

eigenvalue =  7.882167216569541
k =  2
Compute the weights in GS (lowest Aw peak)
 state  10237   d3z2r2 dn 0 0 2   dx2y2 dn 0 0 2   d3z2r2 up 0 0 0   dx2y2 up 0 0 0   px dn -1 0 0 
 S_Ni= 1 ,  Sz_Ni= -1 ,  S_Cu= 1 ,  Sz_Cu= 1 ,  bonding= 1 , weight =  0.04143126324780791 

 state  11433   d3z2r2 dn 0 0 2   dx2y2 dn 0 0 2   d3z2r2 up 0 0 0   dx2y2 up 0 0 0   py dn 0 -1 0 
 S_Ni= 1 ,  Sz_Ni= -1 ,  S_Cu= 1 ,  Sz_Cu= 1 ,  bonding= 1 , weight =  0.04143126324780791 

 state  13830   d3z2r2 up 0 0 2   dx2y2 up 0 0 2   d3z2r2 dn 0 0 0   dx2y2 dn 0 0 0   py dn 0 1 2 
 S_Ni= 1 ,  Sz_Ni= 1 ,  S_Cu= 1 ,  Sz_Cu= -1 ,  bonding= 1 , weight =  0.041431263247807895 

 state  14490   d3z2r2 up 0 0 2   dx2y2 up 0 0 2   d3z2r2 dn 0 0 0   dx2y2 dn 0 0 0   px dn 1 0 2 
 S_Ni= 1 ,  Sz_Ni= 1 ,  S_Cu= 1 ,  Sz_Cu= -1 ,  bonding= 1 , weight =  0.04143126324780789 

 state  12852   dx2y2 up 0 0 2   dx2y2 dn 0 0 2   d3z2r2 dn 0 0 0   dx2y2 dn 0 0 0   apz up 0 0 1 
 S_Ni= 0 ,  Sz_Ni= 0 ,  S_Cu= 1 ,  Sz_Cu= -1 ,  

eigenvalue =  7.882167216569581
k =  3
Compute the weights in GS (lowest Aw peak)
 state  12456   d3z2r2 dn 0 0 2   dx2y2 dn 0 0 2   d3z2r2 up 0 0 0   dx2y2 up 0 0 0   px up -1 0 2 
 S_Ni= 1 ,  Sz_Ni= -1 ,  S_Cu= 1 ,  Sz_Cu= 1 ,  bonding= -1 , weight =  0.04735316220636296 

 state  34218   d3z2r2 up 0 0 2   dx2y2 up 0 0 2   d3z2r2 dn 0 0 0   dx2y2 dn 0 0 0   px up 1 0 0 
 S_Ni= 1 ,  Sz_Ni= 1 ,  S_Cu= 1 ,  Sz_Cu= -1 ,  bonding= -1 , weight =  0.047353162206362376 

 state  23434   d3z2r2 up 0 0 2   dx2y2 up 0 0 2   d3z2r2 dn 0 0 0   dx2y2 dn 0 0 0   py up 0 1 0 
 S_Ni= 1 ,  Sz_Ni= 1 ,  S_Cu= 1 ,  Sz_Cu= -1 ,  bonding= -1 , weight =  0.04735316220634768 

 state  12680   d3z2r2 dn 0 0 2   dx2y2 dn 0 0 2   d3z2r2 up 0 0 0   dx2y2 up 0 0 0   py up 0 -1 2 
 S_Ni= 1 ,  Sz_Ni= -1 ,  S_Cu= 1 ,  Sz_Cu= 1 ,  bonding= -1 , weight =  0.047353162206347506 

 state  23432   d3z2r2 up 0 0 2   dx2y2 dn 0 0 2   d3z2r2 up 0 0 0   dx2y2 up 0 0 0   py dn 0 1 0 
 S_Ni= 1 ,  Sz_Ni= 0 ,  S_Cu= 1 ,  Sz_Cu= 1

...

eigenvalue =  8.948257544769833
k =  16
Compute the weights in GS (lowest Aw peak)
 state  11429   d3z2r2 up 0 0 2   dx2y2 dn 0 0 2   d3z2r2 up 0 0 0   dx2y2 dn 0 0 0   py dn 0 -1 0 
 S_Ni= 1 ,  Sz_Ni= 0 ,  S_Cu= 1 ,  Sz_Cu= 0 ,  bonding= 1 , weight =  0.058299214309627444 

 state  11426   d3z2r2 up 0 0 2   dx2y2 up 0 0 2   d3z2r2 dn 0 0 0   dx2y2 dn 0 0 0   py dn 0 -1 0 
 S_Ni= 1 ,  Sz_Ni= 1 ,  S_Cu= 1 ,  Sz_Cu= -1 ,  bonding= 1 , weight =  0.05829921430962715 

 state  11433   d3z2r2 dn 0 0 2   dx2y2 dn 0 0 2   d3z2r2 up 0 0 0   dx2y2 up 0 0 0   py dn 0 -1 0 
 S_Ni= 1 ,  Sz_Ni= -1 ,  S_Cu= 1 ,  Sz_Cu= 1 ,  bonding= 1 , weight =  0.058299214309626195 

 state  13833   d3z2r2 up 0 0 2   dx2y2 dn 0 0 2   d3z2r2 up 0 0 0   dx2y2 dn 0 0 0   py dn 0 1 2 
 S_Ni= 1 ,  Sz_Ni= 0 ,  S_Cu= 1 ,  Sz_Cu= 0 ,  bonding= 1 , weight =  0.05829921430962503 

 state  13830   d3z2r2 up 0 0 2   dx2y2 up 0 0 2   d3z2r2 dn 0 0 0   dx2y2 dn 0 0 0   py dn 0 1 2 
 S_Ni= 1 ,  Sz_Ni= 1 ,  S_Cu= 1 ,  Sz_Cu= -1 , 

eigenvalue =  8.948257544769836
k =  17
Compute the weights in GS (lowest Aw peak)
 state  11435   d3z2r2 dn 0 0 2   dx2y2 dn 0 0 2   d3z2r2 up 0 0 0   dx2y2 up 0 0 0   py up 0 -1 0 
 S_Ni= 1 ,  Sz_Ni= -1 ,  S_Cu= 1 ,  Sz_Cu= 1 ,  bonding= 1 , weight =  0.03100248994958998 

 state  11432   d3z2r2 up 0 0 2   dx2y2 dn 0 0 2   d3z2r2 up 0 0 0   dx2y2 dn 0 0 0   py up 0 -1 0 
 S_Ni= 1 ,  Sz_Ni= 0 ,  S_Cu= 1 ,  Sz_Cu= 0 ,  bonding= 1 , weight =  0.031002489949582445 

 state  11428   d3z2r2 up 0 0 2   dx2y2 up 0 0 2   d3z2r2 dn 0 0 0   dx2y2 dn 0 0 0   py up 0 -1 0 
 S_Ni= 1 ,  Sz_Ni= 1 ,  S_Cu= 1 ,  Sz_Cu= -1 ,  bonding= 1 , weight =  0.03100248994957443 

 state  13839   d3z2r2 dn 0 0 2   dx2y2 dn 0 0 2   d3z2r2 up 0 0 0   dx2y2 up 0 0 0   py up 0 1 2 
 S_Ni= 1 ,  Sz_Ni= -1 ,  S_Cu= 1 ,  Sz_Cu= 1 ,  bonding= 1 , weight =  0.03100248994954487 

 state  13836   d3z2r2 up 0 0 2   dx2y2 dn 0 0 2   d3z2r2 up 0 0 0   dx2y2 dn 0 0 0   py up 0 1 2 
 S_Ni= 1 ,  Sz_Ni= 0 ,  S_Cu= 1 ,  Sz_Cu= 0 ,  

eigenvalue =  8.948257544769861
k =  18
Compute the weights in GS (lowest Aw peak)
 state  14499   d3z2r2 dn 0 0 2   dx2y2 dn 0 0 2   d3z2r2 up 0 0 0   dx2y2 up 0 0 0   px up 1 0 2 
 S_Ni= 1 ,  Sz_Ni= -1 ,  S_Cu= 1 ,  Sz_Cu= 1 ,  bonding= 1 , weight =  0.026869917556444946 

 state  10232   d3z2r2 up 0 0 2   dx2y2 up 0 0 2   d3z2r2 dn 0 0 0   dx2y2 dn 0 0 0   px up -1 0 0 
 S_Ni= 1 ,  Sz_Ni= 1 ,  S_Cu= 1 ,  Sz_Cu= -1 ,  bonding= 1 , weight =  0.02686991755644488 

 state  14496   d3z2r2 up 0 0 2   dx2y2 dn 0 0 2   d3z2r2 up 0 0 0   dx2y2 dn 0 0 0   px up 1 0 2 
 S_Ni= 1 ,  Sz_Ni= 0 ,  S_Cu= 1 ,  Sz_Cu= 0 ,  bonding= 1 , weight =  0.026869917556442507 

 state  10236   d3z2r2 up 0 0 2   dx2y2 dn 0 0 2   d3z2r2 up 0 0 0   dx2y2 dn 0 0 0   px up -1 0 0 
 S_Ni= 1 ,  Sz_Ni= 0 ,  S_Cu= 1 ,  Sz_Cu= 0 ,  bonding= 1 , weight =  0.026869917556441105 

 state  10239   d3z2r2 dn 0 0 2   dx2y2 dn 0 0 2   d3z2r2 up 0 0 0   dx2y2 up 0 0 0   px up -1 0 0 
 S_Ni= 1 ,  Sz_Ni= -1 ,  S_Cu= 1 ,  Sz_Cu= 1 ,

eigenvalue =  8.94825754476989
k =  19
Compute the weights in GS (lowest Aw peak)
 state  12450   d3z2r2 up 0 0 2   dx2y2 up 0 0 2   d3z2r2 dn 0 0 0   dx2y2 dn 0 0 0   px up -1 0 2 
 S_Ni= 1 ,  Sz_Ni= 1 ,  S_Cu= 1 ,  Sz_Cu= -1 ,  bonding= -1 , weight =  0.041286079523523325 

 state  12453   d3z2r2 up 0 0 2   dx2y2 dn 0 0 2   d3z2r2 up 0 0 0   dx2y2 dn 0 0 0   px up -1 0 2 
 S_Ni= 1 ,  Sz_Ni= 0 ,  S_Cu= 1 ,  Sz_Cu= 0 ,  bonding= -1 , weight =  0.04128607952352039 

 state  12456   d3z2r2 dn 0 0 2   dx2y2 dn 0 0 2   d3z2r2 up 0 0 0   dx2y2 up 0 0 0   px up -1 0 2 
 S_Ni= 1 ,  Sz_Ni= -1 ,  S_Cu= 1 ,  Sz_Cu= 1 ,  bonding= -1 , weight =  0.041286079523515644 

 state  34218   d3z2r2 up 0 0 2   dx2y2 up 0 0 2   d3z2r2 dn 0 0 0   dx2y2 dn 0 0 0   px up 1 0 0 
 S_Ni= 1 ,  Sz_Ni= 1 ,  S_Cu= 1 ,  Sz_Cu= -1 ,  bonding= -1 , weight =  0.04128607952350576 

 state  34220   d3z2r2 up 0 0 2   dx2y2 dn 0 0 2   d3z2r2 up 0 0 0   dx2y2 dn 0 0 0   px up 1 0 0 
 S_Ni= 1 ,  Sz_Ni= 0 ,  S_Cu= 1 ,  Sz_Cu= 0

sumweight= 10.000000000003286
LmLn= 4.722911945161914e-05   bonding0= 4.722911945161914e-05   bonding1= 0.0   bonding-1= 0.0
d8d8L= 4.1779080449427415   bonding0= 0.029135358430280086   bonding1= 1.961876045200202   bonding-1= 2.1868966413122606
d9Ld8L= 0.5615101171203143   bonding0= 0.11136755536474283   bonding1= 0.21218351717137968   bonding-1= 0.23795904458418393
d9L2d8= 0.6831436393689021   bonding0= 0.10918455772992292   bonding1= 0.273838027875407   bonding-1= 0.3001210537635757
d9L3d9= 0.00037978584753361707   bonding0= 0.0003797637130099523   bonding1= 1.1067261832682081e-08   bonding-1= 1.1067261832683957e-08
d9L2d9L= 0.09038773195800653   bonding0= 0.02827376323993394   bonding1= 0.02956680723486542   bonding-1= 0.03254716148320828
s= 0.003316996571973599   bonding0= 0.003316996571973599   bonding1= 0.0   bonding-1= 0.0
d9-O2-L2 + d9L-O2-L + d9L2-O2= 0.04526991844769813   bonding0= 0.0   bonding1= 0.0   bonding-1= 0.0
d9-O-L3 + d9L-O-L2 + d9L2-O-L + d9L3-O= 0.035735792475672

In [2]:
data=np.array([1,4])
row=np.array([1,2])
col=np.array([0,1])
a=(sps.coo_matrix((data,(row,col)),shape=(3,3))/2).toarray()
b=(a.conjugate()).transpose()

print(a)
print(b)
c = a.dot(b)
print(c)

[[0.  0.  0. ]
 [0.5 0.  0. ]
 [0.  2.  0. ]]
[[0.  0.5 0. ]
 [0.  0.  2. ]
 [0.  0.  0. ]]
[[0.   0.   0.  ]
 [0.   0.25 0.  ]
 [0.   0.   4.  ]]


In [3]:
a = ["a","b","c","d","e"]
b= [1,3,2,4]
print(a.index("d"))  #数据的查找

3


In [4]:
list[2]

list[2]

In [5]:

Ni_layer.append(3)
print (Ni_layer)

NameError: name 'Ni_layer' is not defined

In [ ]:
if Ni_layer==[1,2,3,3]:
    print (1)

In [ ]:
lst1 = [1, 11, 111]
lst2 = [2, 22, 222]
lst3 = [3, 33, 333]
lst4 = [4, 44, 444]
lst5 = [5, 55, 555]
lst6 = [6, 66, 666]


def lamlist1(l1, l2):
    '''
    reduce the 'for' circulation
    '''     
    funs = []
    for i in l1: 
        for j in l2: 
            x = lambda i=i, j=j: (i,j)
            funs.append(x)

    return funs    
lamlist1(range(-2,3),[0,1])
